In [113]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
library(lavaanPlot)
library(Deducer)
library(gridExtra)
library(reshape2)
#install.packages('ggsignif')
library(ggsignif)
library(svglite)

In [4]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])

### Exclude outliers in the imaging dataset (n=22k)
di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Mental health issues
di$schizophrenia <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$schizophrenia[di[i]=="Schizophrenia"] <- 1
di$schizophrenia[di[i]=="Depression"] <- 1
di$schizophrenia[di[i]=="Mania, hypomania, bipolar or manic-depression"] <- 1
di$schizophrenia[di[i]=="Bulimia nervosa"] <- 1
di$schizophrenia[di[i]=="Anorexia nervosa"] <- 1
di$schizophrenia[di[i]=="Attention deficit or attention deficit and hyperactivity disorder (ADD/ADHD)"] <- 1
di$schizophrenia[di[i]=="Panic attacks"] <- 1
}

# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i] =="High blood pressure"] <- 1
}



#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
di$excluded[di$bipolar_dis ==1] <- 1
di$excluded[di$personality_dis == 1] <- 1
di$excluded[di$schizophrenia == 1] <- 1
#di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')
nrow(ukbb_all)

# New UKB dataframe with only one BMI timepoint

In [425]:
rm(UKB)
UKB=data.frame(ukbb_all$eid, ukbb_all$body_mass_index_bmi_f21001_2_0,ukbb_all$age_when_attended_assessment_centre_f21003_2_0,
                                                                       ukbb_all$sex_f31_0_0,
                                                                       ukbb_all$volume_of_accumbensarea_left_hemisphere_f26564_2_0,
                                                                       ukbb_all$volume_of_accumbensarea_right_hemisphere_f26595_2_0,
                                                                       ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0,
                  ukbb_all$volume_of_caudate_right_hemisphere_f26590_2_0, 
               ukbb_all$volume_of_caudate_left_hemisphere_f26559_2_0, 
               ukbb_all$volume_of_thalamusproper_right_hemisphere_f26589_2_0, ukbb_all$volume_of_thalamusproper_left_hemisphere_f26558_2_0,
                  ukbb_all$volume_of_pallidum_right_hemisphere_f26592_2_0, ukbb_all$volume_of_pallidum_left_hemisphere_f26561_2_0, 
               ukbb_all$volume_of_putamen_right_hemisphere_f26591_2_0, ukbb_all$volume_of_putamen_left_hemisphere_f26560_2_0,
              ukbb_all$volume_of_accumbens_left_f25023_2_0, ukbb_all$volume_of_accumbens_right_f25024_2_0,
              ukbb_all$volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0, ukbb_all$volume_of_caudate_right_f25014_2_0,
              ukbb_all$volume_of_caudate_left_f25013_2_0, ukbb_all$volume_of_thalamus_right_f25012_2_0,
              ukbb_all$volume_of_thalamus_left_f25011_2_0, ukbb_all$volume_of_pallidum_right_f25018_2_0,
              ukbb_all$volume_of_pallidum_left_f25017_2_0,ukbb_all$volume_of_putamen_right_f25016_2_0,
              ukbb_all$volume_of_putamen_left_f25015_2_0)

colnames(UKB)=c('ID','BMI','Age','Gender','LeftAcc','RightAcc','eTIV','CaudateR','CaudateL','ThalamusR','ThalamusL',
                   'PallidumR','PallidumL','PutamenR','PutamenL','LeftAccFSL','RightAccFSL','ScalingF','CaudateRFSL','CaudateLFSL',
                'ThalamusRFSL','ThalamusLFSL',
                   'PallidumRFSL','PallidumLFSL','PutamenRFSL','PutamenLFSL')


UKB=na.omit(UKB)

UKB$RAccSt=((UKB$RightAcc/UKB$eTIV)-mean(UKB$RightAcc/UKB$eTIV))/sd(UKB$RightAcc/UKB$eTIV)
UKB$LAccSt=((UKB$LeftAcc/UKB$eTIV)-mean(UKB$LeftAcc/UKB$eTIV))/sd(UKB$LeftAcc/UKB$eTIV)
UKB$RAccStFSL=((UKB$RightAccFSL*UKB$ScalingF)-mean(UKB$RightAccFSL*UKB$ScalingF))/sd(UKB$RightAccFSL*UKB$ScalingF)
UKB$LAccStFSL=((UKB$LeftAccFSL*UKB$ScalingF)-mean(UKB$LeftAccFSL*UKB$ScalingF))/sd(UKB$LeftAccFSL*UKB$ScalingF)
UKB$RPutSt=((UKB$PutamenR/UKB$eTIV)-mean(UKB$PutamenR/UKB$eTIV))/sd(UKB$PutamenR/UKB$eTIV)
UKB$LPutSt=((UKB$PutamenL/UKB$eTIV)-mean(UKB$PutamenL/UKB$eTIV))/sd(UKB$PutamenL/UKB$eTIV)
UKB$RPutStFSL=((UKB$PutamenRFSL*UKB$ScalingF)-mean(UKB$PutamenRFSL*UKB$ScalingF))/sd(UKB$PutamenRFSL*UKB$ScalingF)
UKB$LPutStFSL=((UKB$PutamenLFSL*UKB$ScalingF)-mean(UKB$PutamenLFSL*UKB$ScalingF))/sd(UKB$PutamenLFSL*UKB$ScalingF)
UKB$RPalSt=((UKB$PallidumR/UKB$eTIV)-mean(UKB$PallidumR/UKB$eTIV))/sd(UKB$PallidumR/UKB$eTIV)
UKB$LPalSt=((UKB$PallidumL/UKB$eTIV)-mean(UKB$PallidumL/UKB$eTIV))/sd(UKB$PallidumL/UKB$eTIV)
UKB$RPalStFSL=((UKB$PallidumRFSL*UKB$ScalingF)-mean(UKB$PallidumRFSL*UKB$ScalingF))/sd(UKB$PallidumRFSL*UKB$ScalingF)
UKB$LPalStFSL=((UKB$PallidumLFSL*UKB$ScalingF)-mean(UKB$PallidumLFSL*UKB$ScalingF))/sd(UKB$PallidumLFSL*UKB$ScalingF)
UKB$RCauSt=((UKB$CaudateR/UKB$eTIV)-mean(UKB$CaudateR/UKB$eTIV))/sd(UKB$CaudateR/UKB$eTIV)
UKB$LCauSt=((UKB$CaudateL/UKB$eTIV)-mean(UKB$CaudateL/UKB$eTIV))/sd(UKB$CaudateL/UKB$eTIV)
UKB$RCauStFSL=((UKB$CaudateRFSL*UKB$ScalingF)-mean(UKB$CaudateRFSL*UKB$ScalingF))/sd(UKB$CaudateRFSL*UKB$ScalingF)
UKB$LCauStFSL=((UKB$CaudateLFSL*UKB$ScalingF)-mean(UKB$CaudateLFSL*UKB$ScalingF))/sd(UKB$CaudateLFSL*UKB$ScalingF)
UKB$RThaSt=((UKB$ThalamusR/UKB$eTIV)-mean(UKB$ThalamusR/UKB$eTIV))/sd(UKB$ThalamusR/UKB$eTIV)
UKB$LThaSt=((UKB$ThalamusL/UKB$eTIV)-mean(UKB$ThalamusL/UKB$eTIV))/sd(UKB$ThalamusL/UKB$eTIV)
UKB$RThaStFSL=((UKB$ThalamusRFSL*UKB$ScalingF)-mean(UKB$ThalamusRFSL*UKB$ScalingF))/sd(UKB$ThalamusRFSL*UKB$ScalingF)
UKB$LThaStFSL=((UKB$ThalamusLFSL*UKB$ScalingF)-mean(UKB$ThalamusLFSL*UKB$ScalingF))/sd(UKB$ThalamusLFSL*UKB$ScalingF)

variables_for_outliers=c(30:46)
#variables_for_outliers=c(2)

for (j in variables_for_outliers) {
    Q3=as.numeric(quantile(UKB[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(UKB[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    UKB[j][UKB[j]<lower]=NA
    UKB[j][UKB[j]>upper]=NA
}

UKB=na.omit(UKB)e

## Calculate moderations

In [423]:
stats=matrix()
ps=matrix()

vars=c('RAccSt$', 'LAccSt$', 'RAccStFSL$', 'LAccStFSL$', 
       'RPutSt$', 'LPutSt$', 'RPutStFSL$', 'LPutStFSL$', 
       'RPalSt$', 'LPalSt$', 'RPalStFSL$', 'LPalStFSL$',
       'RCauSt$', 'LCauSt$', 'RCauStFSL$', 'LCauStFSL$',
       'RThaSt$', 'LThaSt$', 'RThaStFSL$', 'LThaStFSL$')


for (i in 1:length(vars)) {

a=(summary(lmp(UKB[[grep(vars[i],colnames(UKB))]] ~ 
      (scale((UKB$BMI))) *
      scale(UKB$Age) +
      as.factor(UKB$Gender), center=TRUE, perm='Exact', maxIter=10000)))
    
    stats[i]=a$coefficients[4,3]
    ps[i]=a$coefficients[4,1]

}

stats_adj=p.adjust(stats,method='BH')
stats_adj
ps

## Plot all age moderations

In [202]:
UKB$BMIsl=(log(UKB$BMI)-mean(log(UKB$BMI)))/sd(log(UKB$BMI))
UKB$Ages=scale(UKB$Age)

regions=data.frame(UKB$RAccSt,UKB$LAccSt,UKB$RAccStFSL,UKB$LAccStFSL,
                   UKB$RPutSt,UKB$LPutSt,UKB$RPutStFSL,UKB$LPutStFSL,
                   UKB$RPalSt,UKB$LPalSt,UKB$RPalStFSL,UKB$LPalStFSL,
                   UKB$RCauSt,UKB$LCauSt,UKB$RCauStFSL,UKB$LCauStFSL,
                   UKB$RThaSt,UKB$LThaSt,UKB$RThaStFSL,UKB$LThaStFSL)

names=c('Right accumbens volume - FreeS','Left accumbens volume - FreeS',
        'Right accumbens volume - FSL','Left accumbens volume - FSL',
        'Right putamen volume - FreeS','Left putamen volume - FreeS',
        'Right putamen volume - FSL','Left putamen volume - FSL',
        'Right pallidum volume - FreeS','Left pallidum volume - FreeS',
        'Right pallidum volume - FSL','Left pallidum volume - FSL',
        'Right caudate volume - FreeS','Left caudate volume - FreeS',
        'Right caudate volume - FSL','Left caudate volume - FSL',
        'Right thalamus volume - FreeS','Left thalamus volume - FreeS',
        'Right thalamus volume - FSL','Left thalamus volume - FSL')

tags1=c('c','a','g','e','c','a','g','e','c','a','g','e','c','a','g','e','c','a','g','e')
tags2=c('d','b','h','f','d','b','h','f','d','b','h','f','d','b','h','f','d','b','h','f')
        
filenames=c('Accumbens','Putamen','Pallidum','Caudate','Thalamus')
colours=c("BuPu","BuPu","BuPu","BuPu","YlGn","YlGn","YlGn","YlGn","RdPu","RdPu","RdPu","RdPu",
         "YlOrRd","YlOrRd","YlOrRd","YlOrRd",'Blues','Blues','Blues','Blues')
colours1=c('purple','purple','purple','purple','green3','green3','green3','green3',
           'magenta','magenta','magenta','magenta','goldenrod1','goldenrod1','goldenrod1','goldenrod1',
          'royalblue','royalblue','royalblue','royalblue')

significance=c('','p=0.025','','p<0.001',
              'p=0.022','p<0.001','p=0.017','p<0.001',
              'p<0.001','p<0.001','p<0.001','p<0.001',
              'p<0.001','p<0.001','p=0.041','p=0.039',
              'p<0.001','p<0.001','','')

slopes=list()
agedep=list()

for (i in 1:length(regions)) {
    
    a=as.numeric(regions[[i]])
    Reg=lm(a ~ BMIsl*Age + as.factor(Gender), data = UKB)
    
    ss=sim_slopes(Reg, pred = BMIsl, modx = Age, 
                     modx.values = c(45, 50,55,60,65,70,75,80))
    
    slopes[[i]]=(plot(ss) + 
                 xlim(-0.2,0.2) + 
                 scale_color_manual(values=colours1[i]) + 
                 theme_classic() + 
                 ylab(' ') + 
                 xlab(paste('Slope of BMI','*',names[i], sep=''))) + 
    labs(tag = tags1[i]) + 
    geom_text(x=0.15, y=8, label = significance[i], size=3.5)
    

    agedep[[i]]=(interact_plot(Reg, pred = BMIsl, modx = Age, interval = TRUE, int.width = 0.8, 
                  color.class=colours[i], line.thickness=0.8) + 
                 theme_classic() + 
                 ylab(names[i]) + 
                 xlab('BMI z-scores')) + 
    labs(tag = tags2[i]) + 
    ylim(-1,1)

    
}

p_index=1
for (p in 1:5) {        
    grid=grid.arrange(
      slopes[[p_index]], slopes[[p_index+1]], slopes[[p_index+2]], slopes[[p_index+3]], 
      agedep[[p_index]], agedep[[p_index+1]], agedep[[p_index+2]], agedep[[p_index+3]],
      widths = c(1,1,1,1),
      layout_matrix = rbind(c(2, 6, 1, 5),
                            c(4, 8, 3, 7)))
    
    p_index=p_index+4


    ggsave(
      paste('/dagher/dagher11/filip/OBAL/HBM_Figures/',filenames[p],'.tiff',sep=''),
      plot = grid,
      device = "tiff",
      scale = 1,
      width = 40,
      height = 20,
      units = c('cm'),
      dpi = 300)
    
    }

## Creating and matching obesity maintainers and developers

In [524]:
rm(UKB)
UKB=data.frame(ukbb_all$eid, ukbb_all$body_mass_index_bmi_f21001_2_0,ukbb_all$age_when_attended_assessment_centre_f21003_2_0,
                                                                       ukbb_all$sex_f31_0_0,
                                                                       ukbb_all$volume_of_accumbensarea_left_hemisphere_f26564_2_0,
                                                                       ukbb_all$volume_of_accumbensarea_right_hemisphere_f26595_2_0,
                                                                       ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0,
                  ukbb_all$volume_of_caudate_right_hemisphere_f26590_2_0, 
               ukbb_all$volume_of_caudate_left_hemisphere_f26559_2_0, 
               ukbb_all$volume_of_thalamusproper_right_hemisphere_f26589_2_0, ukbb_all$volume_of_thalamusproper_left_hemisphere_f26558_2_0,
                  ukbb_all$volume_of_pallidum_right_hemisphere_f26592_2_0, ukbb_all$volume_of_pallidum_left_hemisphere_f26561_2_0, 
               ukbb_all$volume_of_putamen_right_hemisphere_f26591_2_0, ukbb_all$volume_of_putamen_left_hemisphere_f26560_2_0,
              ukbb_all$volume_of_accumbens_left_f25023_2_0, ukbb_all$volume_of_accumbens_right_f25024_2_0,
              ukbb_all$volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0, ukbb_all$volume_of_caudate_right_f25014_2_0,
              ukbb_all$volume_of_caudate_left_f25013_2_0, ukbb_all$volume_of_thalamus_right_f25012_2_0,
              ukbb_all$volume_of_thalamus_left_f25011_2_0, ukbb_all$volume_of_pallidum_right_f25018_2_0,
              ukbb_all$volume_of_pallidum_left_f25017_2_0,ukbb_all$volume_of_putamen_right_f25016_2_0,
              ukbb_all$volume_of_putamen_left_f25015_2_0, ukbb_all$body_mass_index_bmi_f21001_0_0)
colnames(UKB)=c('ID','BMI','Age','Gender','LeftAcc','RightAcc','eTIV','CaudateR','CaudateL','ThalamusR','ThalamusL',
                   'PallidumR','PallidumL','PutamenR','PutamenL','LeftAccFSL','RightAccFSL','ScalingF','CaudateRFSL','CaudateLFSL',
                'ThalamusRFSL','ThalamusLFSL',
                   'PallidumRFSL','PallidumLFSL','PutamenRFSL','PutamenLFSL','BMI1')

UKB=na.omit(UKB)

UKB$RAccSt=((UKB$RightAcc/UKB$eTIV)-mean(UKB$RightAcc/UKB$eTIV))/sd(UKB$RightAcc/UKB$eTIV)
UKB$LAccSt=((UKB$LeftAcc/UKB$eTIV)-mean(UKB$LeftAcc/UKB$eTIV))/sd(UKB$LeftAcc/UKB$eTIV)
UKB$RAccStFSL=((UKB$RightAccFSL*UKB$ScalingF)-mean(UKB$RightAccFSL*UKB$ScalingF))/sd(UKB$RightAccFSL*UKB$ScalingF)
UKB$LAccStFSL=((UKB$LeftAccFSL*UKB$ScalingF)-mean(UKB$LeftAccFSL*UKB$ScalingF))/sd(UKB$LeftAccFSL*UKB$ScalingF)
UKB$RPutSt=((UKB$PutamenR/UKB$eTIV)-mean(UKB$PutamenR/UKB$eTIV))/sd(UKB$PutamenR/UKB$eTIV)
UKB$LPutSt=((UKB$PutamenL/UKB$eTIV)-mean(UKB$PutamenL/UKB$eTIV))/sd(UKB$PutamenL/UKB$eTIV)
UKB$RPutStFSL=((UKB$PutamenRFSL*UKB$ScalingF)-mean(UKB$PutamenRFSL*UKB$ScalingF))/sd(UKB$PutamenRFSL*UKB$ScalingF)
UKB$LPutStFSL=((UKB$PutamenLFSL*UKB$ScalingF)-mean(UKB$PutamenLFSL*UKB$ScalingF))/sd(UKB$PutamenLFSL*UKB$ScalingF)
UKB$RPalSt=((UKB$PallidumR/UKB$eTIV)-mean(UKB$PallidumR/UKB$eTIV))/sd(UKB$PallidumR/UKB$eTIV)
UKB$LPalSt=((UKB$PallidumL/UKB$eTIV)-mean(UKB$PallidumL/UKB$eTIV))/sd(UKB$PallidumL/UKB$eTIV)
UKB$RPalStFSL=((UKB$PallidumRFSL*UKB$ScalingF)-mean(UKB$PallidumRFSL*UKB$ScalingF))/sd(UKB$PallidumRFSL*UKB$ScalingF)
UKB$LPalStFSL=((UKB$PallidumLFSL*UKB$ScalingF)-mean(UKB$PallidumLFSL*UKB$ScalingF))/sd(UKB$PallidumLFSL*UKB$ScalingF)
UKB$RCauSt=((UKB$CaudateR/UKB$eTIV)-mean(UKB$CaudateR/UKB$eTIV))/sd(UKB$CaudateR/UKB$eTIV)
UKB$LCauSt=((UKB$CaudateL/UKB$eTIV)-mean(UKB$CaudateL/UKB$eTIV))/sd(UKB$CaudateL/UKB$eTIV)
UKB$RCauStFSL=((UKB$CaudateRFSL*UKB$ScalingF)-mean(UKB$CaudateRFSL*UKB$ScalingF))/sd(UKB$CaudateRFSL*UKB$ScalingF)
UKB$LCauStFSL=((UKB$CaudateLFSL*UKB$ScalingF)-mean(UKB$CaudateLFSL*UKB$ScalingF))/sd(UKB$CaudateLFSL*UKB$ScalingF)
UKB$RThaSt=((UKB$ThalamusR/UKB$eTIV)-mean(UKB$ThalamusR/UKB$eTIV))/sd(UKB$ThalamusR/UKB$eTIV)
UKB$LThaSt=((UKB$ThalamusL/UKB$eTIV)-mean(UKB$ThalamusL/UKB$eTIV))/sd(UKB$ThalamusL/UKB$eTIV)
UKB$RThaStFSL=((UKB$ThalamusRFSL*UKB$ScalingF)-mean(UKB$ThalamusRFSL*UKB$ScalingF))/sd(UKB$ThalamusRFSL*UKB$ScalingF)
UKB$LThaStFSL=((UKB$ThalamusLFSL*UKB$ScalingF)-mean(UKB$ThalamusLFSL*UKB$ScalingF))/sd(UKB$ThalamusLFSL*UKB$ScalingF)

### Create grouping variables

In [525]:
UKB$GroupBMI1=cut(UKB$BMI1,c(0,25,27.5,100))
UKB$GroupBMI3=cut(UKB$BMI,c(0,25,27.5,100))

### Divide into groups

In [526]:
set.seed(904392)
UKB_dfs=UKB %>%
  group_split(GroupBMI1, GroupBMI3)
UKB %>%
  group_by(GroupBMI1,GroupBMI3) %>%
summarize(n())

UKB_dfs=na.omit(UKB_dfs)

In [527]:
Maintain=as.data.frame(UKB_dfs[[9]])
Maintain$Group='Maintainers'
Develop=as.data.frame(rbind(UKB_dfs[[3]]))
Develop$Group='Developers'

MainDev=rbind(Maintain,Develop)
MainDev$Sample <- as.logical(MainDev$Group == 'Developers')

### Match groups

In [528]:
matched_groups <- matchit(Sample ~ BMI + Age + Gender, data = MainDev, method="nearest")
summary(matched_groups, standardize = TRUE)
#plot(matched_groups)

In [529]:
df.match <- match.data(matched_groups)#[1:ncol(MainDev)]

### Plot BMIs

In [14]:
library(tidyr)
gathered=gather(df.match, key='timepoint', value='BMI', c(BMI, BMI1), factor_key=FALSE)

sample_sum <- gathered %>%
  group_by(timepoint, Group) %>%
  summarize(mean = mean(BMI),
            sd   = sd(BMI),
            mean_p2sd = mean + 2 * sd,
            mean_m2sd = mean - 2 * sd) %>%
  ungroup()

sample_sum$timepoint[sample_sum$timepoint=="BMI"] <- 2
sample_sum$timepoint[sample_sum$timepoint=="BMI1"] <- 1
#sample_sum$timepoint[sample_sum$timepoint=="BMI2"] <- 2

#ggplot(data=gathered, aes(x=timepoint, y=BMI, group=Group, colour=Group)) + #geom_point(show.legend = FALSE) + 
#geom_line(show.legend=TRUE)
sample_sum$Group=factor(sample_sum$Group, ordered=TRUE)
sample_sum$timepoint=as.numeric(sample_sum$timepoint)

tiff('BMI_long.tif', units="in", width=6, height=5, res=300)
ggplot(sample_sum, aes(x = (timepoint),  color = Group, fill = Group)) + 
  geom_ribbon(aes(ymin = mean_m2sd, ymax = mean_p2sd), alpha = 0.1) +
  geom_line(aes(y= mean)) +  theme_minimal() + labs(x = "Timepoint") +
scale_fill_manual(values=c("#530077", "#FF21FF")) + scale_color_manual(values=c("#530077", "#FF21FF")) +
ylab(bquote('BMI ['~'kg /'~~m^2~']')) + scale_x_continuous(breaks=c(1,2)) 
dev.off()




p <- ggplot(sample_sum %>%
              gather(stat, BMI, mean, mean_p2sd:mean_m2sd), 
            aes(x = timepoint, y = BMI)) + 
  geom_line(aes(color = Group, linetype = stat))+ facet_wrap(sample_sum$Group)


### Exclude outliers

In [534]:
variables_for_outliers=c(29,31:43)

for (j in variables_for_outliers) { # BMI, Right and left: accumbens, caudate, putamen, pallidum, thalamus, eTIV
    Q3=as.numeric(quantile(df.match[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(df.match[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    df.match[j][df.match[j]<lower]=NA
    df.match[j][df.match[j]>upper]=NA
}

df.match=na.omit(df.match)

### Calculate stats 

In [544]:
stats=matrix(14,1)
ps=matrix()

vars=c('LAccSt$','LAccStFSL$', 
       'RPutSt$', 'LPutSt$', 'RPutStFSL$', 'LPutStFSL$', 
       'RPalSt$', 'LPalSt$', 'RPalStFSL$', 'LPalStFSL$',
       'RCauSt$', 'LCauSt$', 'RCauStFSL$', 'LCauStFSL$')

for (i in 1:length(vars)) {
    
    print(vars[i])
    
    a=(summary(lmp(df.match[[grep(vars[i],colnames(df.match))]] ~ Group + Age + Gender, data = df.match, perm="Exact", 
            seqs=FALSE,center=FALSE, projections = FALSE, qr = TRUE, maxIter=10000)))
    
    stats[i]=a$coefficients[2,3]
    ps[i]=a$coefficients[2,1]
    
    }

stats_adj=p.adjust(stats,method='BH')
vars
stats_adj
ps

In [23]:
Nacc_plot=select(df.match, ID, Group, LAccSt,LAccStFSL)
PutR_plot=select(df.match, ID, Group, RPutSt,RPutStFSL)
PutL_plot=select(df.match, ID, Group, LPutSt,LPutStFSL)
PalR_plot=select(df.match, ID, Group, RPalSt,RPalStFSL)
PalL_plot=select(df.match, ID, Group, LPalSt,LPalStFSL)
CauR_plot=select(df.match, ID, Group, RCauSt,RCauStFSL)
CauL_plot=select(df.match, ID, Group, LCauSt,LCauStFSL)
Nacc_plot=melt(Nacc_plot,id.vars = c("ID",'Group'))
PutR_plot=melt(PutR_plot,id.vars = c("ID",'Group'))
PalR_plot=melt(PalR_plot,id.vars = c("ID",'Group'))
CauR_plot=melt(CauR_plot,id.vars = c("ID",'Group'))
PutL_plot=melt(PutL_plot,id.vars = c("ID",'Group'))
PalL_plot=melt(PalL_plot,id.vars = c("ID",'Group'))
CauL_plot=melt(CauL_plot,id.vars = c("ID",'Group'))

In [24]:
df.match$GRP<-car::recode(df.match$Group, "'Maintainers'='2'; else='1'")


In [548]:
graphs=list()

dframes=c('Nacc_plot', 'PutL_plot', 'PalL_plot', 'CauL_plot','PutR_plot','PalR_plot','CauR_plot')
titles=c('Left nucleus accumbens volume','Left putamen volume','Left pallidum volume',
        'Left caudate volume','Right putamen volume','Right pallidum volume','Right caudate volume')
labels=c('','','0.037','p=0.001','','','','','p=0.043','p=0.004','','p=0.040','','')
colours=c('PuRd','YlGn','RdPu','YlOrRd','YlGn','RdPu','YlOrRd')
lines=c(0.85,1.85,1.15,2.15,0.85,1.85,0.85,1.85,1.15,2.15,0.85,2.15,0.85,1.85)
tags=c('a','b','d','f','c','e','g')
index=1
for (i in 1:length(dframes)) {

        graphs[[i]]=ggplot(eval(as.name(dframes[i])), aes(x=variable, y=value, fill=Group)) + 
            geom_violin(trim=FALSE,position=position_dodge(0.75), width=0.6) +
            geom_point(pch = 21, position = position_jitterdodge(dodge.width = 0.75, jitter.width=0.08), size=0.1) + 
            geom_boxplot(outlier.shape = NA, alpha=0.7, position=position_dodge(0.75), width=0.3) + 
            theme_minimal() +
            ylab(titles[i]) + 
            ylim(-4.5,4.5) +
            xlab('Software package') +
            scale_x_discrete(labels=c("Freesurfer", "FSL")) +
            theme(axis.title.x=element_blank()) +
            scale_fill_brewer(palette=colours[i]) +
            geom_text(x=1, y=4.4, label = labels[index], size=2) +
            geom_text(x=2, y=4.4, label = labels[index+1], size=2) +
            geom_segment(x = 0.85, xend = lines[index], y = 4, yend=4, size=0.05) +
            geom_segment(x = 1.85, xend = lines[index+1], y = 4, yend=4, size=0.05) +
            labs(tag = tags[i])
            
        index=index+2
    }


grid=grid.arrange(
  graphs[[1]], graphs[[2]], graphs[[3]], graphs[[4]], graphs[[5]], graphs[[6]], graphs[[7]],
  widths = c(1,1),
  layout_matrix = rbind(c(1, 1),
                        c(2, 5),
                        c(3, 6),
                        c(4, 7))
)

ggsave(
  '/dagher/dagher11/filip/OBAL/HBM_Figures/MainDev.tiff',
  plot = grid,
  device = "tiff",
  scale = 1,
  width = 20,
  height = 30,
  units = c('cm'),
  dpi = 300
)


In [503]:
structures=c('LAccSt$','LAccStFSL$','LPalSt$','LPalStFSL$','RPalSt$','RPalStFSL$','LPutSt$','LPutStFSL$',
             'RPutSt$','RPutStFSL$','LCauSt$','LCauStFSL$','RCauSt$','RCauStFSL$')
titles=c('Left nucleus accumbens volume - Freesurfer','Left nucleus accumbens volume - FSL',
         'Left pallidum volume - Freesurfer','Left pallidum volume - FSL','Right pallidum volume - Freesurfer',
         'Right pallidum volume - FSL','Left putamen volume - Freesurfer','Left putamen volume - FSL',
         'Right putamen volume - Freesurfer','Right putamen volume - FSL','Left caudate volume - Freesurfer',
         'Left caudate volume - FSL','Right caudate volume - Freesurfer','Right caudate volume - FSL')


In [504]:
for (i in 1:length(structures)) {
        
        graph=ggplot(df.match, aes(x=Group, y=df.match[[grep(structures[i],colnames(df.match))]])) + 
        geom_violin(trim=FALSE) +
        geom_boxplot(outlier.colour="red", outlier.shape=NA, outlier.size=4) + 
        geom_jitter(shape=16, position=position_jitter(0.1)) + 
        theme_minimal() +
        ylab(titles[i]) + 
        ylim(-5,5)
        
        print(graph)
        
}


